<a href="https://colab.research.google.com/github/marcexpositg/CRISPRed/blob/master/02.Model/2.1.DataSimulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.1. Data Simulation

Data is simulated combining the efficiency predictions from Doench et.al 2014 and the outcomes diversity prediction of Indelphi (Shen et al 2018).

Hence, it requires importing the InDelhpi Python module. This module was found in a public GitHub repository at: https://github.com/maxwshen/inDelphi-model
One of the modules was modified, so a modified version is uploaded from the GitHub repository of these work.

In [ ]:


# Previously, the InDelhi module was loaded from drive, but required mounting drive manually.
from google.colab import drive
drive.mount('/content/drive')
%cd "drive/My Drive/CRISPred/02.Model"
#%cd "drive/Shared\ drives/CRISPred/02.Model"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/CRISPred/02.Model


Indelphi requires scikit-learn==0.20.0, so the version available in Google collabs is downgraded to that version.

In [ ]:
!pip install -q progress
!pip install scikit-learn==0.20.0

     |████████████████████████████████| 5.3MB 4.3MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
# This is just a code to test INdelphi

import sys
sys.path.append('./inDelphi-model/')
import inDelphi
inDelphi.init_model(celltype = 'mESC')

left_seq = 'AGAATCGCCCGCGGTCCATCCTTTATCAGCGGGAATTCAAGCGCACCAGCCAGAGGTGTA'
right_seq = 'CCGTGGACGTGAGAAAGAAGAAACATAATATTCGCACTAGATCCATCCCCATACCTGACC'
seq = left_seq + right_seq
cutsite = len(left_seq)

pred_df, stats = inDelphi.predict(seq, cutsite)

print(pred_df)

print(stats)

     Length Genotype position  Predicted frequency Category Inserted Bases
0         5                 4         1.090908e+01      del            NaN
1         7                 5         2.366646e+01      del            NaN
2         7                 7         1.131012e+00      del            NaN
3         8                 3         9.412543e-01      del            NaN
4         8                 6         9.412543e-01      del            NaN
..      ...               ...                  ...      ...            ...
402      59                 e         5.209985e-07      del            NaN
403       1               NaN         1.180538e+01      ins              A
404       1               NaN         1.638573e+00      ins              C
405       1               NaN         1.449101e+00      ins              T
406       1               NaN         8.589149e-01      ins              G

[407 rows x 5 columns]
{'Phi': 1.911503071603351, 'Precision': 0.49365571536045105, '1-bp ins frequ

In [ ]:
import sys


# Import indelphi.
# For local usage:
#sys.path.append('/home/mexposit/bioinf/simdata/inDelphi-model/')
# For google collab:
# not necessary: sys.path.append('drive/My Drive/CRISPred/02.Model/inDelphi-model/')
sys.path.append('./inDelphi-model/')
# for 3rd person: sys.path.append('drive/Shared\ drives/CRISPred/02.Model/inDelphi-model/')
import inDelphi

import csv
import numpy as np
import pandas as pd

from math import exp

from progress.bar import Bar

# Doench 2014 parameters
doench_params = [
# pasted/typed table from PDF and converted to zero-based positions
(1,'G',-0.2753771),(2,'A',-0.3238875),(2,'C',0.17212887),(3,'C',-0.1006662),
(4,'C',-0.2018029),(4,'G',0.24595663),(5,'A',0.03644004),(5,'C',0.09837684),
(6,'C',-0.7411813),(6,'G',-0.3932644),(11,'A',-0.466099),(14,'A',0.08537695),
(14,'C',-0.013814),(15,'A',0.27262051),(15,'C',-0.1190226),(15,'T',-0.2859442),
(16,'A',0.09745459),(16,'G',-0.1755462),(17,'C',-0.3457955),(17,'G',-0.6780964),
(18,'A',0.22508903),(18,'C',-0.5077941),(19,'G',-0.4173736),(19,'T',-0.054307),
(20,'G',0.37989937),(20,'T',-0.0907126),(21,'C',0.05782332),(21,'T',-0.5305673),
(22,'T',-0.8770074),(23,'C',-0.8762358),(23,'G',0.27891626),(23,'T',-0.4031022),
(24,'A',-0.0773007),(24,'C',0.28793562),(24,'T',-0.2216372),(27,'G',-0.6890167),
(27,'T',0.11787758),(28,'C',-0.1604453),(29,'G',0.38634258),(1,'GT',-0.6257787),
(4,'GC',0.30004332),(5,'AA',-0.8348362),(5,'TA',0.76062777),(6,'GG',-0.4908167),
(11,'GG',-1.5169074),(11,'TA',0.7092612),(11,'TC',0.49629861),(11,'TT',-0.5868739),
(12,'GG',-0.3345637),(13,'GA',0.76384993),(13,'GC',-0.5370252),(16,'TG',-0.7981461),
(18,'GG',-0.6668087),(18,'TC',0.35318325),(19,'CC',0.74807209),(19,'TG',-0.3672668),
(20,'AC',0.56820913),(20,'CG',0.32907207),(20,'GA',-0.8364568),(20,'GG',-0.7822076),
(21,'TC',-1.029693),(22,'CG',0.85619782),(22,'CT',-0.4632077),(23,'AA',-0.5794924),
(23,'AG',0.64907554),(24,'AG',-0.0773007),(24,'CG',0.28793562),(24,'TG',-0.2216372),
(26,'GT',0.11787758),(28,'GG',-0.69774)]

def calc_doench_score(seq):
    """
    Addapted FROM CRISPOR!! This is the Doench, 2014 score. Not the 2016.
    Code reproduced following paper's methods section. Thanks to Daniel McPherson for fixing it.
    Input is a 30mer: 4bp 5', 20bp guide, 3bp PAM, 3bp 5'
    Output is a gRNA activity score, scaled from 0 (low efficiency) to 100 (high efficiency).
    """
    intercept = 0.59763615
    gc_high = -0.1665878
    gc_low = -0.2026259

    assert(len(seq)==30)
    score = intercept

    grna_seq = seq[4:24]
    gc_count = grna_seq.count("G") + grna_seq.count("C")
    if gc_count <= 10:
        gc_weight = gc_low
    if gc_count > 10:
        gc_weight = gc_high
    score += abs(10-gc_count)*gc_weight

    for pos, model_seq, weight in doench_params:
        sub_seq = seq[pos:pos+len(model_seq)]
        if sub_seq == model_seq:
            score += weight
    exp_score = int(100*(1.0/(1.0+exp(-score))))

    return exp_score

# input data, define cutsite and number of reads
# all gRNAs must be in the same orientation and position (if not, doench score would fail)

# For local: #target_regions_file = 'C3H_targets.csv'
# For Google Collabs, smaller example (3 sequences only):
target_regions_file = 'SimData/C3H_targets_example.csv'

cutsite = 60
sim_reads = 5000

# #target_seqs = pd.read_csv('target_regions.csv', names=['gRNA_id', 'target_seq'])
target_seqs_data = dict(csv.reader(open(target_regions_file, 'r')))

sim_data = pd.DataFrame()
sim_info = pd.DataFrame()

bar = Bar('Simulating sequences:', max=len(target_seqs_data))

for gRNA_id, target_seq in target_seqs_data.items():

    # Calculate activity score using doench for that guide
    ## 30mer: 4bp 5', 20bp guide, 3bp PAM, 3bp 5'
    seq_for_doench = target_seq[ cutsite - 21 : cutsite + 9]
    doench_score = calc_doench_score(seq_for_doench)
    ## doench score is from 0 to 100, scale to get numb of edited reads
    n_edit_seqs = round(doench_score * sim_reads / 100) #round to have an integer number of reads

    # Calculate editing outcomes
    inDelphi.init_model(celltype = 'mESC')
    pred_df, stats = inDelphi.predict(target_seq, cutsite)
    pred_df = inDelphi.add_mhless_genotypes(pred_df,stats)
    #pred_df = inDelphi.add_genotype_column(pred_df,stats)
    ## adds gaps in the deletions, use add_genotype_column to avoid gaps, but sequences could be confused
    pred_df = inDelphi.add_genotype_column_wgaps(pred_df,stats)
    pred_frequency = np.array(pred_df["Predicted frequency"])
    # normalize probabilities to sum exactly 1
    pred_frequency /= pred_frequency.sum()

    # Simulate data
    ## first, create the edited reads
    edit_seqs = np.random.choice(pred_df["Genotype"], p=pred_frequency, size=(n_edit_seqs))
    ## add non edited reads up to the sim_reads objective
    wt_seqs = np.repeat(target_seq, sim_reads-n_edit_seqs)
    ## merge both and keep in dataframe
    sim_seqs = np.append(edit_seqs,wt_seqs)
    sim_data[gRNA_id] = sim_seqs

    # Keep useful information about simulation
    ## pred_df contains summary of outcomes, but rows are in different order for each simulation
    ## stats has many important parameters
    ## append doench score from 0 to 1, and store in dataframe
    stats['doench_eff_score'] = doench_score
    sim_info[gRNA_id] = pd.Series(stats)

    # Next progress bar
    bar.next()

bar.finish()
# For local:
#sim_data.to_csv('sim_data_grna_lib_ex.csv', index=None)
#sim_info.to_csv('sim_info_grna_lib_ex.csv')
# For google collabs:
sim_data.to_csv('SimData/sim_data_grna_lib_ex.csv', index=None)
sim_info.to_csv('SimData/sim_info_grna_lib_ex.csv')
print("This is the file used to quantify the outcomes:")
print(sim_data)
print("This is just a file with information about the simulation:")
print(sim_info)

This is the file used to quantify the outcomes:
                              ENSMUSG00000033788_gR434r  ...                           ENSMUSG00000033788_gR70f
0     CTCCTAAAGATTGATAATGGTGACAGCTAAACTTCCCCCACTATGG...  ...  CTGACTAACGTTGAATATCGATAAGACACAGAGAAGAGGGGTGGGG...
1     CTCCTAAAGATTGATAATGGTGACAGCTAAACTTCCCCCACTATGG...  ...  CTGACTAACGTTGAATATCGATAAGACACAGAGAAGAGGGGTGGGG...
2     CTCCTAAAGATTGATAATGGTGACAGCTAAACTTCCCCCACTATGG...  ...  CTGACTAACGTTGAATATCGATAAGACACAGAGAAGAGGGGTGGGG...
3     CTCCTAAAGATTGATAATGGTGACAGCTAAACTTCCCCCACTATGG...  ...  CTGACTAACGTTGAATATCGATAAGACACAGAGAAGAGGGGTGGGG...
4     CTCCTAAAGATTGATAATGGTGACAGCTAAACTTCCCCCACTATGG...  ...  CTGACTAACGTTGAATATCGATAAGACACAGAGAAGAGGGGTGGGG...
...                                                 ...  ...                                                ...
4995  CTCCTAAAGATTGATAATGGTGACAGCTAAACTTCCCCCACTATGG...  ...  CTGACTAACGTTGAATATCGATAAGACACAGAGAAGAGGGGTGGGG...
4996  CTCCTAAAGATTGATAATGGTGACAGCTAAACTTCCCCCACTATGG... 